In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from IMLCV.configs.config_general import config

# config(
#     env="local",
#     local_ref_threads=2,
#     max_threads_local=10,
# )

N_TRAIN = 32

config(
    account="2024_117",
    singlepoint_nodes=4,
    default_on_threads=False,
    cpu_cluster="cpu_milan_rhel_9",
    training_cores=N_TRAIN,
)

# config(

#     singlepoint_nodes=4,
#     walltime="12:00:00",
#     bootstrap=False,
#     training_cores=32,

# )

ROOT_DIR=PosixPath('/dodrio/scratch/projects/2024_026/IMLCV/src/IMLCV')
node
hortense
setting python env for hortense
executor='work_queue'
wall_time_s=14160.0


/dodrio/scratch/projects/2024_026/IMLCV/micromamba/envs/py312/lib/python3.12/subprocess.py:1893: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(


setting python env for hortense
executor='work_queue'
wall_time_s=172560.0
setting python env for hortense
executor='work_queue'
wall_time_s=14160.0


/dodrio/scratch/projects/2024_026/IMLCV/micromamba/envs/py312/lib/python3.12/subprocess.py:1893: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
/dodrio/scratch/projects/2024_026/IMLCV/micromamba/envs/py312/lib/python3.12/subprocess.py:1893: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(


In [3]:
from pathlib import Path

from IMLCV.base.rounds import Rounds
from IMLCV.examples.example_systems import CsPbI3_MACE_lattice
from IMLCV.scheme import Scheme

folder = Path("perovskites") / "CsPbI3_cell_002"

if folder.exists():
    scheme = Scheme(rounds=Rounds.create(folder=folder, copy=False, new_folder=False))

else:
    md, refs = CsPbI3_MACE_lattice()

    scheme = Scheme.from_refs(
        mde=md,
        refs=refs,
        folder=folder,
        steps=1000,
    )

In [4]:
from IMLCV.base.UnitsConstants import angstrom, boltzmann, kelvin, kjmol

300 * kelvin * boltzmann / kjmol

2.494341595946812

In [5]:
# scheme.rounds.unzip_cv(cv=2)

In [6]:
# chunk_size = 100

# n = [15, 10, 6][scheme.bias.collective_variable.n - 1]

# print(f"{n=}")

# max_bias = 100 * kjmol
# samples_per_bin = 200
# min_samples_per_bin = 5

# n_max = (2 * n) ** (scheme.bias.collective_variable.n)

# print(f"{n=} {n_max=}")

In [7]:
from molmod.units import angstrom, kjmol

r_cut = 6.0 * angstrom

steps = 1000
chunk_size = None
macro_chunk = N_TRAIN * 16  # sampes per worker
macro_chunk_nl = N_TRAIN * 128
samples_per_bin = 100
min_samples_per_bin = 10
T_Scale = 10
koopman = True
eps = 0.15  # 10 percent overlap
max_bias = 100 * kjmol
num_cv_rnds = 2
lag_n = 30
koopman_wham = True
n_max_descriptors = 1
l_max_descriptor = 1
alpha_rematch = 0.2
ncv = 3
min_cv = 2
direct_bias = False

max_cv_basis_fun = 2000

bias_num_points = 5e4
cv_num_points = 5e4
num_sample_rounds = 5


n_max = 1e3  # 30**3

max_blocks = 400

In [8]:
l_max_descriptor

1

In [9]:
def update_cv():
    import jax
    import jax.numpy as jnp

    from IMLCV.base.CV import CV, CvTrans
    from IMLCV.implementations.CV import LatticeInvariants2, _cv_index, get_sinkhorn_divergence_2, sb_descriptor
    from IMLCV.implementations.CvDiscovery import TransformerMAF

    print("getting descriptor")

    descriptor = sb_descriptor(
        r_cut=r_cut,
        n_max=n_max_descriptors,
        l_max=l_max_descriptor,
        reshape=True,
        reduce=True,
    )

    rounds = scheme.rounds

    dlo_0 = rounds.data_loader(
        cv_round=0,
        start=0,
        weight=False,
        new_r_cut=r_cut,
        time_series=True,
        lag_n=20,
    )

    # print(f"{dlo.nl=} {dlo.sp=}")

    print("computing soap descriptor")

    cv_0, cv_0_t = dlo_0.apply_cv(
        descriptor,
        x=dlo_0.sp,
        x_t=dlo_0.sp_t,
        nl=dlo_0.nl,
        nl_t=dlo_0.nl,
        macro_chunk=macro_chunk,
        verbose=True,
    )

    print("sinkhorn div")
    tr = get_sinkhorn_divergence_2(
        nli=dlo_0.nl,
        pi=CV.stack(*[a[-1] for a in cv_0]),
        alpha_rematch=alpha_rematch,
        jacobian=False,
    )

    cv_1, cv_1_t = dlo_0.apply_cv(
        tr,
        x=cv_0,
        x_t=cv_0_t,
        nl=dlo_0.nl,
        nl_t=dlo_0.nl,
        macro_chunk=macro_chunk,
        verbose=True,
    )

    # cv_2 = cv_1
    # cv_2_t=cv_1_t

    print("computing nonzero elems")

    # cv_2_stack = CV.stack(*cv_2)

    # idx = jnp.argwhere(jnp.std(cv_2_stack.cv,axis=0) > 1e-10)

    # @jax.vmap
    # def f(x):
    #     return jnp.ravel_multi_index(x, cv_2_stack.shape[1:], mode="wrap")

    # idx = f(idx)

    # print(f"{idx.shape=}")

    # sl = CvTrans.from_cv_function(_cv_index, indices=idx)

    # cv_3, cv_3_t = dlo_0.apply_cv(
    #     sl,
    #     x=cv_,
    #     x_t=cv_2_t,
    #     macro_chunk=macro_chunk,
    #     verbose=True,
    # )

    km = dlo_0.koopman_model(
        cv_0=cv_1,
        cv_tau=cv_1_t,
        method="tcca",
        calc_pi=False,
        add_1=True,
        trans=None,
        chunk_size=chunk_size,
        macro_chunk=macro_chunk,
        verbose=True,
        out_dim=-1,
        symmetric=True,
        correlation=True,
        eps=1e-8,
        eps_pre=1e-8,
        max_features_pre=500,
    )

    from IMLCV.base.rounds import DataLoaderOutput

    mask = km.s > 0.1

    o = km.W0.T

    o = o[:, mask]

    print(f"{jnp.sum(mask)=}")

    km_tr = CvTrans.from_cv_function(
        DataLoaderOutput._transform,
        static_argnames=["add_1", "add_1_pre"],
        add_1=False,
        add_1_pre=False,
        q=o,
        pi=km.cov.pi_0,
        argmask=km.argmask,
    )

    if km.trans is not None:
        km_tr = km.trans * km_tr

    tot_flow = (descriptor * tr * km_tr) + LatticeInvariants2

    print(f"testing comp")

    cv_out, cv_out_t = dlo_0.apply_cv(
        tot_flow,
        x=dlo_0.sp,
        x_t=dlo_0.sp_t,
        nl=dlo_0.nl,
        nl_t=dlo_0.nl,
        macro_chunk=macro_chunk,
        verbose=True,
    )

    print(f"{cv_out[0].shape=}")

    start = 1

    if scheme.rounds.cv == 0:
        start = 0

    if scheme.rounds.cv == 1:
        start = 1

    scheme.update_CV(
        dlo_kwargs=dict(
            out=cv_num_points,
            num_cv_rounds=1,
            time_series=True,
            new_r_cut=r_cut,
            num=num_cv_rnds + 1,
            start=start,
            lag_n=lag_n,
            split_data=False,
            chunk_size=chunk_size,
            only_finished=True,
            T_scale=T_Scale,
            macro_chunk=macro_chunk,
            samples_per_bin=samples_per_bin,
            min_samples_per_bin=min_samples_per_bin,
            macro_chunk_nl=macro_chunk_nl,
            verbose=True,
            n_max=n_max,
            weight=True,
            only_update_nl=True,
            # reweight_to_fes=True,
            reweight_inverse_bincount=True,
            scale_times=False,
        ),
        transformer=TransformerMAF(
            outdim=ncv,
            descriptor=tot_flow,
            pre_scale=False,
            correct_bias=True,
            use_ground_bias=True,
            T_scale=T_Scale,
            koopman_weighting=False,
            method="tcca",
            solver="eig",
            max_features=max_cv_basis_fun,
            max_features_pre=max_cv_basis_fun,  # if scheme0.rounds.cv > 0 else 100,
            trans=None,
            eps=1e-8,
            eps_pre=1e-8,
        ),
        chunk_size=chunk_size,
        plot=True,
        new_r_cut=r_cut,
        save_samples=True,
        save_multiple_cvs=False,
        test=False,
        percentile=10,  # get 90.0% of the points
        max_bias=max_bias,
        macro_chunk=macro_chunk,
        verbose=True,
        n_max=n_max,
    )

In [10]:
import jax.numpy as jnp


def sample(eps=0.1):
    cv_dim = scheme.rounds.get_collective_variable().n

    n_umbrella = int(jnp.floor(max_blocks ** (1 / cv_dim)))

    if n_umbrella > 20:
        n_umbrella = 20

    print(f"n_umbrella: {n_umbrella}")

    scheme.inner_loop(
        n=n_umbrella,
        rnds=num_sample_rounds if scheme.rounds.cv > 1 else 3,
        steps=1000,
        init=0,
        eps_umbrella=eps,
        fes_bias_rnds=num_cv_rnds,
        chunk_size=chunk_size,
        macro_chunk=macro_chunk,
        samples_per_bin=samples_per_bin,
        min_samples_per_bin=min_samples_per_bin,
        max_bias=max_bias,
        n_max_fes=n_max,
        convergence_kl=0.05,
        thermolib=False,
        T_scale=T_Scale,
        koopman=koopman,
        koopman_wham=koopman_wham,
        out=bias_num_points,
        direct_bias=direct_bias,
        first_round_without_bias=scheme.rounds.cv == 1,
    )

In [11]:
def get_fes_bias():
    bias = scheme.FESBias(
        plot=True,
        samples_per_bin=samples_per_bin,
        min_samples_per_bin=min_samples_per_bin,
        chunk_size=chunk_size,
        macro_chunk=macro_chunk,
        num_rnds=num_cv_rnds,
        vmax=max_bias,
        max_bias=max_bias,
        only_finished=True,
        n_max=n_max,
        thermolib=False,
        out=bias_num_points,
        T_scale=T_Scale,
        # divide_by_histogram=True,
        lag_n=lag_n,
        koopman_wham=koopman_wham,
        koopman=koopman,
        direct_bias=direct_bias,
    )

    scheme.rounds.add_round(bias=bias)

In [ ]:
if scheme.rounds.cv == 0:
    update_cv()

for i in range(10):
    sample(eps=eps)
    update_cv()

n_umbrella: 20
cv_round=1
i_0=1
running round i=1 with 1000 steps
running first round wihtout biases


Process WorkQueue-Submit-Process:
Process WorkQueue-Submit-Process:
Process WorkQueue-Submit-Process:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/dodrio/scratch/projects/2024_026/IMLCV/micromamba/envs/py312/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/dodrio/scratch/projects/2024_026/IMLCV/micromamba/envs/py312/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/dodrio/scratch/projects/2024_026/IMLCV/micromamba/envs/py312/lib/python3.12/site-packages/parsl/process_loggers.py", line 26, in wrapped
    r = func(*args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^
  File "/dodrio/scratch/projects/2024_026/IMLCV/micromamba/envs/py312/lib/python3.12/site-packages/parsl/executors/workqueue/executor.py", line 861, in _work_queue_submit_wait
    task = task_queue.get(timeout=1)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

In [ ]:
get_fes_bias()

estimating bias from koopman Theory!
